# Summary of interacting cis-eQTL analysis

In [ ]:
import functools
import pandas as pd

## Functions

### Cached functions

In [ ]:
@functools.lru_cache()
def get_mashr_degs(feature, tissue):
    df = pd.read_csv("../../_m/%s/lfsr_feature_4tissues.txt.gz" % feature, 
                     sep='\t').loc[:, ["Effect", tissue]]
    return df[(df[tissue] < 0.05)].rename(columns={tissue: "lfsr"})


@functools.lru_cache()
def annotate_degs(feature, tissue):
    config = {
        "genes": "/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/_m/caudate/gene_annotation.tsv",
        "transcripts": "/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/_m/caudate/tx_annotation.tsv", 
        "exons": "/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/_m/caudate/exon_annotation.tsv",
        "junctions": "/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/_m/caudate/jxn_annotation.tsv",
    }
    annot = pd.read_csv(config[feature], sep='\t').loc[:, ["names", "seqnames", "gencodeID"]]
    return get_mashr_degs(feature, tissue).merge(annot, left_on="Effect", 
                                                 right_on="names").drop(["names"], axis=1)


### Simple functions

In [ ]:
def extract_features(tissue):
    ## Extract significant eQTL using mashr
    genes = annotate_degs("genes", tissue)
    trans = annotate_degs("transcripts", tissue)
    exons = annotate_degs("exons", tissue)
    juncs = annotate_degs("junctions", tissue)
    return genes, trans, exons, juncs


def output_summary(tissue, variable):
    ## Extract eQTL using mashr
    genes, trans, exons, juncs = extract_features(tissue)
    ## Total significant eQTLs
    gg = len(set(genes[variable]))
    tt = len(set(trans[variable]))
    ee = len(set(exons[variable]))
    jj = len(set(juncs[variable]))
    print("\nGene:\t\t%d\nTranscript:\t%d\nExon:\t\t%d\nJunction:\t%d" % 
          (gg, tt, ee, jj)) 
    
    
def get_DEGs_result_by_tissue(tissue):
    genes, trans, exons, juncs = extract_features(tissue)
    genes["Type"] = "Gene"
    trans["Type"] = "Transcript"
    exons["Type"] = "Exon"
    juncs["Type"] = "Junction"
    df = pd.concat([genes, trans, exons, juncs])
    df["Type"] = df.Type.astype("category").cat.reorder_categories(["Gene", "Transcript", "Exon", "Junction"])
    df["Tissue"] = tissue.replace(".", " ")
    return df

## Feature Summary

### Summarize results mashr (local false sign rate < 0.05)

In [ ]:
for tissue in ["Caudate", "Dentate.Gyrus", "DLPFC", "Hippocampus"]:
    print("")
    print(tissue)
    ## significant Features
    print("\nFeatures")
    output_summary(tissue, "Effect")
    print("\nGeneid")
    ## significant Geneid
    output_summary(tissue, "gencodeID")

In [ ]:
caud8 = get_DEGs_result_by_tissue("Caudate")
gyrus = get_DEGs_result_by_tissue("Dentate.Gyrus")
dlpfc = get_DEGs_result_by_tissue("DLPFC")
hippo = get_DEGs_result_by_tissue("Hippocampus")

## Save significant results

In [ ]:
pd.concat([caud8, gyrus, dlpfc, hippo])\
  .sort_values(["Tissue", "Type", "lfsr"])\
  .loc[:, ["Tissue", "Effect", "gencodeID", "seqnames", "lfsr", "Type"]]\
  .to_csv("BrainSeq_ancestry_4features_4regions.txt.gz", sep='\t', index=False)